In [77]:
import pandas as pd
import numpy as np

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure, show
#from bokeh.sampledata.stocks import AAPL
days_per_year = 52 * 5

In [78]:
hist=pd.read_csv("data/archive/ETFs/spy.us.txt")
print(hist)

            Date    Open    High     Low   Close    Volume  OpenInt
0     2005-02-25  104.77  106.00  104.68  105.79  70221808        0
1     2005-02-28  105.55  105.68  104.56  105.08  79695344        0
2     2005-03-01  105.22  105.87  105.22  105.62  54607412        0
3     2005-03-02  105.21  106.22  105.10  105.57  73733090        0
4     2005-03-03  105.99  106.20  105.15  105.61  71286823        0
...          ...     ...     ...     ...     ...       ...      ...
3196  2017-11-06  258.30  259.00  258.22  258.85  49529148        0
3197  2017-11-07  258.97  259.35  258.11  258.67  57218448        0
3198  2017-11-08  258.47  259.22  258.15  259.11  50277699        0
3199  2017-11-09  257.73  258.39  256.36  258.17  94537282        0
3200  2017-11-10  257.73  258.21  257.37  258.09  59862364        0

[3201 rows x 7 columns]


In [79]:
dates = np.array(hist['Date'], dtype=np.datetime64)
date_close_lut = dict(date=dates, close=hist['Close'])
source = ColumnDataSource(data=date_close_lut)

p = figure(height=700,
            width=1900, 
            tools="xpan", 
            toolbar_location=None,
            x_axis_type="datetime", 
            x_axis_location="above",
           background_fill_color="#efefef", 
           x_range=(dates[1500], dates[2500])
           )

p.line('date', 'close', source=source)
p.yaxis.axis_label = 'Price'

select = figure(
    title="Drag the middle and edges of the selection box to change the range above",
    height=200,
    width=1900,
    y_range=p.y_range,
    x_axis_type="datetime",
    y_axis_type=None,
    tools="",
    toolbar_location=None,
    background_fill_color="#efefef"
)

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('date', 'close', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool



In [80]:
def future_below_yr(years, df):
    # dataset_1 = "%Y-%m-%d"
    td = years * days_per_year
    return pd.DataFrame(
            [
                {
                    "date": df['date'][i], 
                    "close": df['close'][i]
                }
                for i in range(len(df)) 
                if df['close'][i] < df['close'][max(0, i-years)] 
            ]
        )

In [81]:
p.circle(
    'date', 
    'close', 
    source=future_below_yr(10, date_close_lut), 
    size=200, 
    color="navy", 
    alpha=0.5
)
print(type(hist))

<class 'pandas.core.frame.DataFrame'>


In [82]:
show(column(p, select))